In [1]:
import os
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model, Input
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
%matplotlib inline

In [2]:
X_train = np.load("128_128_train.npy")
y_train = np.load("train_labels.npy")
X_val = np.load("128_128_val.npy")
y_val = np.load("val_labels.npy")
X_test = np.load("128_128_test.npy")
y_test = np.load("test_labels.npy")

In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
input_shape = X_val[0,:,:,:].shape
model_input = Input(shape=input_shape)

In [5]:
pre_trained_model = VGG16(input_shape=(128, 128, 3), include_top=False, weights="imagenet")


In [6]:
for layer in pre_trained_model.layers:
    print(layer.name)
    layer.trainable = False
    
print(len(pre_trained_model.layers))

input_2
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool
19


In [7]:
last_layer = pre_trained_model.get_layer('block5_pool')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 4, 4, 512)


In [8]:
# Flatten the output layer to 1 dimension
x = layers.GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(7, activation='softmax')(x)

# Configure and compile the model

model = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

c:\Users\muham\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [9]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [10]:
train_datagen = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)

In [11]:
batch_size = 64
epochs = 3
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size), 
                              validation_steps=(X_val.shape[0] // batch_size))

<ipython-input-11-86fe2f61745c>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),


Epoch 1/3
126/126 [==============================] - 404s 3s/step - loss: 1.2468 - accuracy: 0.6315 - val_loss: 1.0760 - val_accuracy: 0.6518
Epoch 2/3
126/126 [==============================] - 404s 3s/step - loss: 1.0612 - accuracy: 0.6706 - val_loss: 0.9680 - val_accuracy: 0.6719
Epoch 3/3
126/126 [==============================] - 367s 3s/step - loss: 0.9952 - accuracy: 0.6788 - val_loss: 0.9339 - val_accuracy: 0.6853


In [12]:
for layer in model.layers[:15]:
    layer.trainable = False

for layer in model.layers[15:]:
    layer.trainable = True

In [13]:
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

In [14]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [15]:
learning_rate_reduction = ReduceLROnPlateau(monitor='acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=3)

In [16]:
batch_size = 64
epochs = 20
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size), callbacks=[learning_rate_reduction])

<ipython-input-16-aa0e431c43d0>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),


Epoch 1/20
126/126 [==============================] - 437s 3s/step - loss: 0.8823 - acc: 0.6929 - val_loss: 0.7503 - val_acc: 0.7344 - lr: 1.0000e-04
Epoch 2/20
126/126 [==============================] - 435s 3s/step - loss: 0.7610 - acc: 0.7315 - val_loss: 0.7328 - val_acc: 0.7232 - lr: 1.0000e-04
Epoch 3/20
126/126 [==============================] - 437s 3s/step - loss: 0.7121 - acc: 0.7451 - val_loss: 0.7495 - val_acc: 0.7098 - lr: 1.0000e-04
Epoch 4/20
126/126 [==============================] - 435s 3s/step - loss: 0.6815 - acc: 0.7562 - val_loss: 0.6615 - val_acc: 0.7455 - lr: 1.0000e-04
Epoch 5/20
126/126 [==============================] - 433s 3s/step - loss: 0.6287 - acc: 0.7711 - val_loss: 0.6863 - val_acc: 0.7388 - lr: 1.0000e-04
Epoch 6/20
126/126 [==============================] - 444s 4s/step - loss: 0.6194 - acc: 0.7752 - val_loss: 0.7086 - val_acc: 0.7411 - lr: 1.0000e-04
Epoch 7/20
126/126 [==============================] - 447s 4s/step - loss: 0.6019 - acc: 0.7851 - va

In [17]:
loss_test, acc_test = model.evaluate(X_test, y_test, verbose=1)
loss_val, acc_val = model.evaluate(X_val, y_val, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))
model.save("model_vgg16.h5")

29/29 [==============================] - 41s 1s/step - loss: 0.6692 - acc: 0.7805
Validation: accuracy = 0.780488  ;  loss_v = 0.669171
Test: accuracy = 0.792415  ;  loss = 0.699120
